In [362]:
import pandas as pd
import numpy as np

In [363]:
countriesSoFar = ['FRA','DEU','BEL','NGA','NLD','CIV','POL']

In [364]:
def printList(pList):
    """Tired of having to type this"""
    for l in pList:
        print(l)

In [365]:
def printListList(pList):
    """Tired of having to type this"""
    for l in pList:
        for ll in l:
            print(ll)

In [178]:
def pullItEm(file):
    dataset = pd.read_csv(file)
    dSet = []
    tempList = []
    numRows = len(dataset)
    for n in range(numRows):
        for line in dataset.iloc[n,:5]:
            tempList.append(line)
        dSet.append(tempList)
        tempList = []
    return dSet

In [366]:
def pullItCom(file):
    """pulls in the Comdata file"""
    dataset = pd.read_csv(file)
    dSet = []
    tempList = []
    numRows = len(dataset)
    for n in range(numRows):
        for line in dataset.iloc[n,:7]:
            tempList.append(line)
        dSet.append(tempList)
        tempList = []
    return dSet

In [173]:
def fixYear(dSet):
    """Fixes the year on the emdat records"""
    postSet = []
    tmpEntity = []
    for d in dSet:
        tmpEntity.append(d[0][-4:])
        tmpEntity.append(d[1])
        tmpEntity.append(d[2])
        tmpEntity.append(d[3])
        tmpEntity.append(d[4])
        postSet.append(tmpEntity)
        tmpEntity = []
    return postSet

In [420]:
def groupByYear(dSet):
    """combines disasters of all type to provide single year record for each country"""
    tmpCountry = dSet[0][1]
    tmpDeathSum = 0
    tmpDollarSum = 0
    tmpYear = dSet[0][0]
    tmpEntry =[]
    dSetR1 =[]

    for i in range(len(dSet)):

        if (dSet[i][0] == tmpYear) and (dSet[i][1] == tmpCountry):
            tmpDeathSum += dSet[i][3]
            tmpDollarSum += dSet[i][4]
        elif (dSet[i][0] !=tmpYear):
            tmpEntry.append(tmpYear)
            tmpEntry.append(tmpCountry)
            tmpEntry.append(tmpDeathSum)
            tmpEntry.append(tmpDollarSum)
            dSetR1.append(tmpEntry)
            tmpEntry = []
            tmpCountry = dSet[i][1]
            tmpDeathSum = dSet[i][3]
            tmpDollarSum = dSet[i][4]
            tmpYear = dSet[i][0]
                
    return(dSetR1)


In [138]:
def calcImpactScore(dSet):
    tmpEntry =[]
    tmpCountry = ""
    tmpYear = 0
    tmpDeath = 0
    AdjDollar = 0
    calcImpact = 0
    dSetR2 = []
    for d in dSet:
        tmpYear = d[0]
        tmpCountry = d[1]
        tmpDeath = d[2]
        if(d[3]!=0):
            AdjDollars = (np.log10(d[3]))/10
        else:
            AdjDollars = d[3]
        calcImpact = AdjDollars +tmpDeath
        tmpEntry.append(tmpYear)
        tmpEntry.append(tmpCountry)
        tmpEntry.append(tmpDeath)
        tmpEntry.append(AdjDollars)
        tmpEntry.append(calcImpact)
        dSetR2.append(tmpEntry)
        tmpEntry = []
    return dSetR2

In [139]:
def getComList(country,commodity):
    """Enter parameter in format 'ISO' """
    country = 'datasets/'+country+'-'+commodity+'.csv'
    return pullItCom(country)

In [141]:
def removeFirstComEntry(comList):
    """gets rid of countries whose adjusted trends are missing from comTrade"""
    editedList = []
    tmpEntry = []
    
    for c in comList:
        for l in c:
            if (str(l[4])==str(l[5]) and str(l[5])=='nan'):
                #print("edit out")
                pass
            else:
                tmpEntry.append(l)
        editedList.append(tmpEntry)
        tmpEntry = []
        
    return editedList

In [206]:
def getComLists(countries,commodity):
    """be sure to use parameter in format ['ISO1','ISO2,...,'ISOn']"""
    l = len(countries)
    comLists = []
    for c in range(l):
        country = countries[c]
        comLists.append(getComList(country,commodity))
    return removeFirstComEntry(comLists)

In [143]:
def getDeathRate():
    file = open('datasets/deathrate.txt','r')
    rawLines = []
    tmpList = []
    deathList = []
    tmpEntry = []
    tmpString = ""
    for line in file:
        rawLines.append(line.strip())
    for l in rawLines:
        if (l[:3] in countriesSoFar):
            for char in range(len(l)):
                if l[char] ==":":
                    tmpEntry.append(tmpString)
                    tmpString = ""
                if l[char] == "/":
                    tmpEntry.append(tmpString.lstrip(':'))          
                    tmpString = ""
                if char == len(l)-1:
                    tmpString += l[char]
                    tmpEntry.append(tmpString.lstrip('/'))               
                    tmpString = "" 
                else:
                    tmpString += l[char]
            tmpList.append(tmpEntry)
            tmpEntry = []

    return tmpList

In [160]:
def calcDeathRate():
    numDeadList = []
    tmpEntry = []
    deathList = getDeathRate()
    for d in deathList:
        pop = int(d[1])
        rate = float(d[2])
        tmpEntry.append(d[0])
        tmpEntry.append(((pop/1000)*rate)*10)
        numDeadList.append(tmpEntry)
        tmpEntry = []
    return numDeadList

In [161]:
def applyDeathRate(emDat,deathList):
    tmpList = []
    tmpEntry = []
    dr = 1
    for n in emDat:
        for d in deathList:
            if (d[0] == n[1]):
                dr = d[1]
        tmpEntry.append(n[0])
        tmpEntry.append(n[1])
        tmpEntry.append(n[2]/dr)
        tmpEntry.append(n[3])
        
        tmpList.append(tmpEntry)
        tmpEntry = []
    return tmpList

In [199]:
def processEmdat():
    emDat = pullItEm('datasets/emdata.csv')
    emDat = fixYear(emDat)
    emDat = groupByYear(emDat)
    deathRate = calcDeathRate()
    emDat = applyDeathRate(emDat,deathRate)
    emDat = calcImpactScore(emDat)
    return emDat

In [258]:
def combineComEm(comList,emDat):
    titleString = ['YEAR','ISO','COMMODITY', 'TREND', 'ADJUSTED TREND', 'ADJUSTED DEATH', 'ADJUSTED $ LOSS', 'CIS' ]
    tmpEntry = []
    tmpList = []
    tmpList.append(titleString)
    for country in comList:
        comName = country[0][1]
        comComm = country[0][2]
        comExpo = country[0][3]
        comTrnd = country[0][4]
        comAdTd = country[0][5]
        for comLine in country:
            comYear = str(comLine[0])
            for e in emDat:
                emYear = e[0]
                emName = e[1]
                emDeath = e[2]
                emDollar = e[3]
                emImp = e[4]
                if (e[0] == comYear and comName == emName):
                    #print('match')
                    tmpEntry.append(comYear)
                    tmpEntry.append(comName)
                    tmpEntry.append(comComm)
                    tmpEntry.append(comExpo)
                    tmpEntry.append(comTrnd)
                    tmpEntry.append(comAdTd)
                    tmpEntry.append(emDeath)
                    tmpEntry.append(emDollar)
                    tmpEntry.append(emImp)
                    tmpList.append(tmpEntry)
                    tmpEntry = []

    return tmpList

In [303]:
def doTheDamnThingHR(countries,commodity):
    comList = getComLists(countries,commodity)
    emDat = processEmdat()
    result = combineComEm(comList,emDat)
    return result

## Next up: 
#### 1) need to incorperate applyDeathRate and calcImpactScore.
#### 2) integrate comList and emDat
#### 3) generate HR file and model input file


In [282]:
record = doTheDamnThingHR(countriesSoFar,'1701')
printList(record)

['YEAR', 'ISO', 'COMMODITY', 'TREND', 'ADJUSTED TREND', 'ADJUSTED DEATH', 'ADJUSTED $ LOSS', 'CIS']
['1995', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 2.5637438975638643e-06, 0.5845098040014257, 0.5845123677453232]
['1996', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 8.011699679887076e-07, 0.37788744720027395, 0.37788824837024193]
['1997', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 3.685381852748055e-06, 0.4, 0.40000368538185277]
['1998', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 2.4035099039661228e-06, 0.5176091259055682, 0.5176115294154722]
['1999', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 2.339416306527026e-05, 0.7097284734298384, 0.7097518675929037]
['2000', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 1.7625739295751568e-06, 0, 1.7625739295751568e-06]
['2001', 'FRA', 1701, 2813245440, 1.137616145, 0.1399516857, 2.723977891161606e-06, 0.5121723945637366, 0.5121751185416278]
['2002', 'FRA', 1701, 2813245440, 1.137616145, 0.13995

In [315]:
def doTheDamnThingMDL(countries,commodity):
    record = doTheDamnThingHR(countries,commodity)
    numCountries = (-1)
    currCountry = ""
    tmpList = []
    tmpHeaders = []
    tmpEntry = []
    for line in record:
         if (line[1] != currCountry):
                numCountries += 1
                currCountry = line[1]
                if (currCountry != 'ISO'):
                    tmpHeaders.append(currCountry)
    tmpHeaders.append('Impact Score')
    tmpHeaders.append('Adjusted Trend')
    tmpList.append(tmpHeaders)
    for line in record:
        if (record.index(line) != 0):
            for h in tmpHeaders[:6]:
                if (line[1] == h):
                    tmpEntry.append(1)
                else:
                    tmpEntry.append(0)
            tmpEntry.append(line[7])
            tmpEntry.append(line[4])
            tmpList.append(tmpEntry)
            tmpEntry = []
        
    return tmpList

In [316]:
y = doTheDamnThingMDL(countriesSoFar,'1701')

In [318]:
printList(y)

['FRA', 'DEU', 'NGA', 'NLD', 'CIV', 'POL', 'Impact Score', 'Adjusted Trend']
[1, 0, 0, 0, 0, 0, 0.5845098040014257, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.37788744720027395, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.4, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.5176091259055682, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.7097284734298384, 1.137616145]
[1, 0, 0, 0, 0, 0, 0, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.5121723945637366, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.6075911761482777, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.677085569208826, 1.137616145]
[1, 0, 0, 0, 0, 0, 0, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.4, 1.137616145]
[1, 0, 0, 0, 0, 0, 0, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.5397940008672037, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.49030899869919436, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.6505149978319906, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.6758154621967389, 1.137616145]
[1, 0, 0, 0, 0, 0, 0, 1.137616145]
[1, 0, 0, 0, 0, 0, 0, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.5816241299991783, 1.137616145]
[1, 0, 0, 0, 0, 0, 0.5685741738602264, 1.13761614

In [320]:
def writeThisBitchOut(dSet,filename):
    file = open(filename,'w')
    tmpString = ""
    for line in dSet:
        for l in line:
            tmpString += string(l)
            tmpString += ','
        tmpString.rstrip(',')
        file.write(tmpString)

In [321]:
writeThisBitchOut(y,'datasets/modelOut.csv')

NameError: name 'string' is not defined